In [1]:
from common import *
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
import json, copy, operator

In [2]:
TEST = True; PROD = False

In [3]:
I = lambda x: x
logavg = lambda x,y: math.expm1((math.log1p(x)+math.log1p(y))/2)
keepSigFig = lambda n: lambda x: round(x, -int(math.floor(math.log10(abs(x)))) + (n - 1)) if x else x

In [4]:
frzset = lambda x: x if isinstance(x, frozenset) else frozenset(x)
pydict = lambda x: O.pycopy(x) if isinstance(x, O) else copy.deepcopy(x)

# #!#!#!#!#!#!#!#!#!#!#! Save System #!#!#!#!#!#!#!#!#!#!#!

## File System

In [68]:
class IndexFileSystem():
    def __init__(self, directory, key):
        self.dir = directory if isinstance(directory, Path) else Path(directory)
        self.dir.mkdir(exist_ok=True, parents=True)
        assert isinstance(key, str), "key must be string"
        self.key = key
        self.io = O()

    def iterIndices(self):
        for dot_params in self.dir.glob('*.' + self.key):
            yield int(dot_params.stem)

    def getFilePath(self, name, *, i):
        return self.dir / (str(i) + '.' + name)

    @staticmethod
    def readWrapper(read):
        '''wraps io read operations to safely return None if file does not exist'''
        @wraps(read)
        def read_safely(*a, **k):
            try:
                return read(*a, **k)
            except FileNotFoundError:
                return None
        return read_safely

    def assignIO(self, name, *, read, write, format='custom'):
        assert format in ['bytes', 'text', 'custom'], "argument `format` must be one of 'bytes' or 'text' or 'custom'"
        if format in ['bytes', 'text']:
            self.io[name] = O(
                read = self.readWrapper( lambda *,i: read(getattr(self.getFilePath(name, i=i), 'read_'+format)()) ),
                write = lambda x,*,i: getattr(self.getFilePath(name, i=i), 'write_'+format)(write(x))
            )
        elif format == 'custom':
            def readCustom(*, i):
                file = self.getFilePath(name, i=i)
                if not file.exists():
                    return None
                try:
                    return read(file)
                except Exception:
                    return read(str(file))
            def writeCustom(*, i):
                file = self.getFilePath(name, i=i)
                try:
                    write(x, file=file)
                except Exception:
                    write(x, file=str(file))
            self.io[name] = O(read=readCustom, write=writeCustom)

class IndexDataStore():    
    def __init__(self, file, _factory_=False):
        if not _factory_:
            assert False, "Cannot instantiate IndexDataStore normally. Please use factory static method."
        self.file = file
        self.op = O()
        self.lists = O()
        self.key = None
        self.keyFunc = lambda keyVal, *, client=False: object()
        self.tbl = {}
        self.nextIndex = 0
        
    def load(self):
        indices = sorted(self.file.iterIndices())
        n = self.nextIndex = (max(indices) if indices else -1) + 1
        for name, _ in dict.items(self.lists):
            self.lists[name] = [None] * self.nextIndex
        
        for i in indices:
            for name in self.lists:
                if self.op[name].load:
                    self.op[name].load(i=i)
                
        if self.key is not None:
            self.assignKey(self.key, self.keyFunc)
            
    def i(self, key, dry=False, client=True, keep=True, save=True):
        keyVal = self.keyFunc(key, client=client)
        
        if keyVal not in self.tbl and not dry:
            for name in self.lists:
                assert len(self.lists[name]) == self.nextIndex, "idk internal error"
                self.lists[name].append(None)
            self.tbl[keyVal] = self.nextIndex
            if save and self.op[self.key].save:
                self.op[self.key].save(key, i=self.nextIndex, keep=keep)
            elif keep and self.op[self.key].keep:
                self.op[self.key].keep(key, i=self.nextIndex, client=client)
            self.nextIndex += 1
        elif keyVal not in self.tbl and dry:
            return None
        
        return self.tbl[keyVal]
            
    def save(self, keep=True, **kwargs):
        '''`client` is always True'''
        assert ('i' in kwargs) ^ (self.key in kwargs), "call to `save` must include exactly one of i= or the key name ="
        if 'i' in kwargs:
            i = kwargs['i']
            del kwargs['i']
        else:
            i = self.i(kwargs[self.key])
        
        for name in kwargs:
            if name == self.key: # already saved in `self.i(save=True)` above
                continue
            if name not in self.op:
                raise AssertionError(f"given save item '{name}' does not have data store ops initialized")
            if self.op[name].save:
                self.op[name].save(kwargs[name], i=i, keep=keep)
            
    def get(self, name, **kwargs):
        assert len(kwargs) == 1 and ('i' in kwargs or self.key in kwargs), "must give assigned 'key' or i"
        if 'i' in kwargs:
            i = kwargs['i']
            del kwargs['i']
        else:
            i = self.i(kwargs[self.key], dry=True)
        if i is None:
            return None
        got = self.lists[name][i]
        if got is None and self.op[name].load:
            got = self.op[name].load(i=i)
        return got
        
    ##################### INSTANCE BUILDING METHODS ##################### : 
    def assignKey(self, name, func):
        assert name in self.lists, "key name must first be assigned ops"
        self.key = name
        self.keyFunc = func
        self.tbl = {self.keyFunc(x): i for i,x in enumerate(self.lists[self.key]) if x is not None}
        
    def assignOperations(self, name, *, load=I, save=I, keep=I, keepSaved=None, keepClient=None):
        if keepClient is None:
            keepClient = keep
            
        #NOTE #TODE? the method names below conflict with the local vars up here, and current Python syntax takes
        # the variables up here. Should this change the code below will stop working. But this is the "nicest" way to do it
        class the(O()):
            def load(*, i, keep=True):
                if not load:
                    return None
                read = self.file.io[name].read(i=i)
                x = load(read) if read is not None else None
                if keep and self.op[name].keep:
                    self.lists[name][i] = self.op[name].keep(x, i=i)
                return x
            
            def save(x, *, i, keep=True):
                if not save:
                    return
                saved_x = save(x)
                self.file.io[name].write(saved_x, i=i)
                if keep and self.op[name].keep:
                    try:
                        self.lists[name][i] = self.op[name].keep(saved_x, i=i, saved=True)
                        assert self.lists[name][i] is not None
                    except (TypeError, AssertionError):
                        self.lists[name][i] = self.op[name].keep(x, i=i, client=True)
                
            def keep(x, *, i, saved=False, client=False):
                if x is None:
                    return None
                assert not (saved and client), "only one of `saved` and `client` can be specified"
                if not any([keep, keepSaved, keepClient]):
                    return None
                kept = None
                if saved and keepSaved:
                    kept = keepSaved(x)
                elif client and keepClient:
                    kept = keepClient(x)
                elif not saved and not client and keep:
                    kept = keep(x)
                if kept is not None:
                    self.lists[name][i] = kept
                return kept
            
        if not load:
            the.load = False
        if not save:
            the.save = False
        if not any([keep, keepSaved, keepClient]):
            the.keep = False
            
        self.op[name] = the
        if name not in self.lists:
            self.lists[name] = [None] * self.nextIndex
    
    
@staticmethod
def __IndexDataStore__from_specs(specs, **kwargs):
    for k, v in kwargs.items():
        specs[k] = v
    
    f = IndexFileSystem(specs.dir, specs.key)
    d = IndexDataStore(f, _factory_=True)
    
    _readwriteformat = {'read', 'write', 'format'}
    _loadsavekeep = {'load', 'save', 'keep', 'keepSaved', 'keepClient'}
    for name, val in dict.items(specs.op):
        f.assignIO(name, **{a: b for a,b in dict.items(val) if a in _readwriteformat})
        d.assignOperations(name, **{a: b for a,b in dict.items(val) if a in _loadsavekeep})
        
    keyFuncClient = specs.keyFuncClient if 'keyFuncClient' in specs else specs.keyFunc
    keyFunc = lambda keyVal, *, client=False: keyFuncClient(keyVal) if client else specs.keyFunc(keyVal)
    d.assignKey(specs.key, keyFunc)
    
    d.load()
    return d
IndexDataStore.from_specs = __IndexDataStore__from_specs


def json_default(o):
    if isinstance(o, np.int64):
        return int(o)  
    raise TypeError
    
class __IndexDataStore__SpecsHelper(metaclass=staticclass):
    def json(op):
        op.format = 'text'
        op.read = json.loads
        op.write = lambda x: json.dumps(x, default=json_default)
        return op
    def pickle(op):
        op.format = 'bytes'
        op.read = pickle.loads
        op.write = pickle.dumps
        return op
IndexDataStore.SpecsHelper = __IndexDataStore__SpecsHelper
IDSSH = IndexDataStore.SpecsHelper

## Model Manager

In [69]:
class ModelManager():
    features_dir = '.features'
    samples_dir = '.samples'
    models_dir = 'models'
    
    def __init__(self, directory, f_specs=None, s_specs=None, p_specs=None): #GLOBAL FDS SDS PDS
        global FDS, SDS, PDS
        if f_specs is None:
            f_specs = FDS
        if s_specs is None:
            s_specs = SDS
        if p_specs is None:
            p_specs = PDS
        self.p_specs = p_specs
        self.dir = directory if isinstance(directory, Path) else Path(directory)
        self.dir.mkdir(exist_ok=True) # no parents because this is user facing and enabling parents could get gnarly
        self.F = IndexDataStore.from_specs(f_specs, dir=self.dir/self.features_dir)
        self.S = IndexDataStore.from_specs(s_specs, dir=self.dir/self.samples_dir)
        (self.dir/self.models_dir).mkdir(exist_ok=True)
        self.load()
        
    def load(self):
        self.pms = {}
        for fi, si in self.iterIndices():
            self.make_params_manager(i=(fi, si))
        
    def iterIndices(self):
        for fdir in (self.dir/self.models_dir).iterdir():
            try:
                fi = int(fdir.name)
            except ValueError:
                continue
            assert fdir.is_dir(), "folder in models folder whose name is just a number must be a folder"
            assert fi < self.F.nextIndex, "features folder found with greater index than what has been labelled"
            for sdir in fdir.iterdir():
                try:
                    si = int(sdir.name)
                except ValueError:
                    continue
                assert sdir.is_dir(), "folder in models folder 1 layer down whose name is just a number must be a folder"
                assert si < self.S.nextIndex, "samples folder found with greater index than what has been labelled"
                yield (fi, si)
        
    def make_params_manager(self, features=None, samples=None, i=None):
        assert (
            not ((features is None) ^ (samples is None)) and
            ((features is not None and samples is not None) ^ (i is not None)),
            "invalid arguments to `make_param_manager`"
        )
        if i is None:
            i = self.F.i(features), self.S.i(samples)
        fi, si = i
        assert 0 <= fi < self.F.nextIndex and 0 <= si < self.S.nextIndex, "bad arguments to `make_params_manager`"
        pm = IndexDataStore.from_specs(self.p_specs, dir=self.dir/self.models_dir/str(fi)/str(si))
        self.pms[fi, si] = pm
        return pm
    
    def i(features, samples): # purely convenience public interface
        return self.F.i(features), self.S.i(samples)
        
    def PM(self, features, samples, get_i=False):
        fi, si = self.F.i(features), self.S.i(samples)
        if get_i:
            return self.iPM(fi, si), (fi, si)
        else:
            return self.iPM(fi, si)
        
    def iPM(self, fi, si):
        if (fi, si) not in self.pms:
            self.make_params_manager(i=(fi, si))
        return self.pms[fi, si]

## specs for managers

In [70]:
lgbNullDataset = lgb.Dataset(pd.DataFrame({'_a_': np.arange(88), '_b_': np.arange(88)}))

### Param search $\textbf{SAVING}$ specs

In [71]:
class PDS(O()):
    '''data transformation code inside `op` object:

    client --save(+write)--> disk; client --keep--> memory; disk --load(+read)--> memory

    read = text/bytes stream -> as-is object read from file
    write = object to save as is to file -> text/bytes stream
    load = as-is object read from file -> object to be loaded in memory
    save = raw object given by client -> object to save as-is to file
    keep = as-is object read from file -> object to keep in memory
    keepClient = raw object given by client -> object to keep in memory
    '''

    key = 'Params'
    keyFunc = lambda params: tuple(sorted(dict.items(params)))

    class op(O()):
        @IDSSH.json
        class Params(O()):
            '''just a dict of the parameter value assignments'''
            keepClient = dict
            
        @IDSSH.json
        class Results(O()):
            '''should be a dict-like of various things, most importantly including "score"'''
            load = lambda x: O(**x)
            save = pydict
            keepClient = lambda x: O.mycopy(x) if isinstance(x, O) else O(**x)
            
        @IDSSH.pickle
        class Training(O()):
            '''tuple (aligning with samples training/cv split tuple) of LightGBM training eval DataFrames'''
            keep = False
            
        @IDSSH.pickle
        class Boosters(O()):
            '''the actual lgb.Booster model. well, a tuple of them, one for each cv set'''
            #read = lambda file: lgb.Booster(model_file=file)
            load = lambda x: tuple(lgb.Booster(train_set=lgbNullDataset).model_from_string(s, verbose=False) for s in x)
            #write = lambda x, file: x.save_model(file)
            save = lambda x: tuple(b.model_to_string() for b in x)
            keep = False

### feature selection / days sampling $\textbf{SAVING}$ specs

In [72]:
class FDS(O()):
    key = 'Feats'
    keyFunc = lambda features: features # internally already coverted to immutable
    keyFuncClient = frzset
    class op(O()):
        '''container of feature name strings'''
        @IDSSH.json
        class Feats(O()):
            load = frozenset
            save = sorted
            keepClient = frozenset
    
class SDS(O()):
    key = 'Samps'
    keyFunc = lambda samples: samples # internally already converted immutable
    keyFuncClient = lambda samples: tuple((frzset(tr), frzset(cv)) for tr, cv in samples)
    class op(O()):
        @IDSSH.json
        class Samps(O()):
            '''tuple (num samples) of 2-tuples of tr/cv containers of the canonical index values for the split'''
            load = lambda samples: tuple((frzset(tr), frzset(cv)) for tr, cv in samples)
            save = lambda samples: [[sorted(tr), sorted(cv)] for tr, cv in samples]
            keepClient = load
        @IDSSH.json
        class Ctor(O()):
            '''should be O specs object'''
            save = pydict
            keepClient = save

# ~!~!~!~!~!~! Model (features/samples/parameters) Search ~!~!~!~!~!~!~!

## parameter search logic

In [73]:
class ParamSearch():
    mix = staticmethod(logavg)
    
    def __init__(self, specs):
        self.specs = O.mycopy(specs)
        self.setup_specs()
        
    def setup_specs(self):
        dsc = self.specs.Discrete
        dsc.keys = list(flatten( dsc.enc.keys() ))
        dsc.assigns = [list(flatten(x)) for x in product(* dsc.enc.values() )]
        obo = self.specs.OneByOne
        for k,v in dict.items(obo.data):
            for i,x in dict.items(obo.default):
                if i not in v:
                    v[i] = x
    
    def search(self):
        dsc, obo = self.specs.Discrete, self.specs.OneByOne
        for assign in dsc.assigns:
            params = dict(zip(dsc.keys, assign))
            coroutine = self.one_by_one()
            for addon in coroutine:
                params.update(addon)
                coroutine.send((yield copy.deepcopy(params))); assert (yield) is None
        
        
    def one_by_one(self):
        obod = self.specs.OneByOne.data
        
        #! main algorithm ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        
        # initialize loop variables
        params = {k: (v.b if 'b' in v else v.a[1]) for k,v in dict.items(obod)}
        ranges = {k: v.a for k,v in dict.items(obod)}
        scores = {k: [-np.inf, -np.inf] for k in params}
        isdone = {k: False for k in params}
        
        # pre loop one-off work
        base_score = yield params; assert (yield) is None
        
        # loop
        for i in range(999_999_999):
            #! try new parameter values ############################## part A of loop work
            
            # initialize local loop variable
            new_scores = {k: [-np.inf, -np.inf] for k in params}
            new_params = {k: ( obod[k].cast(self.mix(v[0], v[1])),
                               obod[k].cast(self.mix(v[1], v[2])) ) for k,v in dict.items(ranges)}
            
            # finish condition check
            isdone = {k: v or i>=obod[k].lim for k,v in dict.items(isdone)}
            if all(isdone.values()):
                break
                
            # try new parameter values for all parameters
            for key in list(params):
                if i >= obod[key].lim:
                    continue
                orig = params[key]
                params[key] = new_params[key][0]
                scores[key][0] = yield params; assert (yield) is None
                params[key] = new_params[key][1]
                scores[key][1] = yield params; assert (yield) is None
                params[key] = orig
            
            #! start setting up values for next loop ######################## part B of loop work
            
            # set params to the best found and see if it betters score, updating ranges also
            #CODE num_nochange = 0
            for key in list(params):
                if scores[key][0] > base_score and scores[key][0] >= scores[key][1]:
                    params[key] = new_params[key][0]
                    ranges[key] = [ranges[key][0], params[key], ranges[key][1]]
                elif scores[key][1] > base_score and scores[key][1] >= scores[key][0]:
                    params[key] = new_params[key][1]
                    ranges[key] = [ranges[key][1], params[key], ranges[key][2]]
                else:
                    ranges[key] = [new_params[key][0], ranges[key][1], new_params[key][1]]
                    #num_nochange += 1
                
            # send out new params
            #CODE if num_nochange < len(params):
            base_score = yield params; assert (yield) is None

## Features / Samples(train/cv split) search logic

#### the Kaggle metric

In [74]:
class KaggleMetric():
    def __init__(self, incr=0):
        self.incr = incr
    
    def attach(self, ms):
        L, s = ms._L, ms._s
        for Ltr, Lcv, tr, cv in zip(L.tr, L.cv, s.tr, s.cv):
            Ltr.timeFactor = ms.Y.time[tr].factorize()[0]
            Lcv.timeFactor = ms.Y.time[cv].factorize()[0]
            Ltr.value = (ms.Y.upDown1*ms.Y.absVal1)[tr]
            Lcv.value = (ms.Y.upDown1*ms.Y.absVal1)[cv]
            Ltr.i = 0
            Lcv.i = 0
    
    def __call__(self, preds, valid_data):
        df_time = valid_data.timeFactor
        #labels = valid_data.get_label()
        values = valid_data.value
        #assert len(labels) == len(df_time)

        preds = preds*2-1
        #labels = labels*2-1
        x_t = preds * values

        # Here we take advantage of the fact that `labels` (used to calculate `x_t`)
        # is a pd.Series and call `group_by`
        x_t_sum = x_t.groupby(df_time).sum()
        score = x_t_sum.mean() / x_t_sum.std()

        valid_data.i += self.incr
        return 'kaggle', score+valid_data.i, True

### Model searching logic

In [75]:
from sklearn.model_selection import GroupShuffleSplit, GroupKFold
        
class ModelSearch():
    
    def __init__(self, specs, mm, *, X, Y, log=None):
        self.specs = specs
        global LOG
        self.log = log if log is not None else LOG
        self.mm = mm
        assert (X.index == Y.index).all()
        self.X = X
        self.Y = Y
        self.setup_specs()
        
    def setup_specs(self):
        self.specs.Samples.data = []
        for code in self.specs.Samples.enc:
            if code.method == 'group.2':
                tr, cv = set(code.data[0]), set(code.data[1])
                tr, cv = self.Y[code.groups].isin(tr), self.Y[code.groups].isin(cv)
                tr, cv = self.Y.index[tr], self.Y.index[cv]
                self.specs.Samples.data += [((tr, cv), (cv, tr))]
            elif code.method == 'GroupShuffleSplit.2':
                tr, cv = next(GroupShuffleSplit(**code.kwargs).split(self.X, self.Y, groups=self.Y[code.groups]))
                tr, cv = self.Y.index[tr], self.Y.index[cv]
                self.specs.Samples.data += [((tr, cv), (cv, tr))]
#             elif code.method == 'GroupKFold':
#                 self.specs.Samples.data.append(tuple(GroupKFold(**code.kwargs)
#                                                      .split(self.X, self.Y, groups=self.Y[code.groups])))
            else:
                assert False, f'sampling method "{code.method}" not implemented'
        
    def walk(self):
        for feats, (samps, sampsEnc) in product(self.specs.Features.data,
                                                zip(self.specs.Samples.data, self.specs.Samples.enc)):
            best_score = -np.inf
            pm, i = self.mm.PM(feats, samps, get_i=True) # this also saves feats and samps
            self._pm = pm
            self.mm.S.save(Samps=samps, Ctor=sampsEnc) #TODO maybe refactor this saving Ctor part out to somewhere better?
            
            self.setup_training(feats, samps, i=i)
            search = ParamSearch(self.specs.search)
            loop = search.search()
            for params in loop: # params are deepcopied out, so can safely save them as-is!
                paramsUse = dict(**self.specs.Params.data, **params)
                results = pm.get('Results', Params=params)
                if not results:
                    self.log('training...')
                    self.train(paramsUse) # sets some state attributes in self: self._save
                    results = self._save.Results
                    if results['score'] > best_score:
                        self._best = (dict(**params), O(**self._save)) #TODO I still copy params here to be safe (needed)?
                    del self._save.Training, self._save.Boosters
                    pm.save(Params=params, **self._save)
                loop.send(results['score'])
                yield i, results
                #del self._save #TODO probably wanna uncomment this in production
            pm.save(Params=self._best[0], **self._best[1])
                
    def run(self):
        for _ in self.walk():
            pass
                
    def setup_training(self, feats, samps, *, i=None):
        samps = self.mm.S.get('Samps', i=i[1]) # this `samps` is a tuple of tuple of frozensets
        _X = self.X[feats]
        #_dummy = pd.Series(range(len(_X)), index=_X.index)
        _s = self._s = O()
        _s.tr, _s.cv = tuple(_X.index.isin(s[0]) for s in samps), tuple(_X.index.isin(s[1]) for s in samps)
        lgb_data_info = dict(
            feature_name = list(_X.columns),
            categorical_feature = list(_X.dtypes[_X.dtypes.isin([np.int64,np.int32])].index),
            free_raw_data = False,
        )
        _L = self._L = O()
        _L.tr = [lgb.Dataset(_X[tr], P[self.specs.model.target][tr], **lgb_data_info,
                            **({'weight': P[self.specs.model.weight][tr]} if 'weight' in self.specs.model else {}))
                for tr in _s.tr]
        _L.cv = [lgb.Dataset(_X[cv], P[self.specs.model.target][cv], reference=Ltr, **lgb_data_info,
                            **({'weight': P[self.specs.model.weight][cv]} if 'weight' in self.specs.model else {}))
                for cv, Ltr in zip(_s.cv, _L.tr)]
        m = self.specs.metric
        if hasattr(m, 'attach'):
            m.attach(self)
        #TODO implement both logloss and Kaggle metric, and stop only when both don't improve in whatever num rounds
        
    def train(self, params):
        def iter_samples():
            for Ltr, Lcv in zip(self._L.tr, self._L.cv):
                evals_result = {}
                # fucking LightGBM deletes 'num_iterations' from `params` after training, like WTF???
                bst = lgb.train(dict(params), Ltr, valid_sets=[Ltr, Lcv], valid_names=['tr', 'cv'],
                          feval=self.specs.metric, evals_result=evals_result, verbose_eval=False)
                df_results = (pd.DataFrame(evals_result['tr']), pd.DataFrame(evals_result['cv']))
                yield bst, df_results
        bsts, dfs = zip(*iter_samples())
        
        class save(O()):
            Training = dfs
            Boosters = bsts
            class Results(O()):
                train = tuple(dft[0].iloc[-1-params['early_stopping_round'], 0] for dft in dfs)
                scores = tuple(dft[1].iloc[-1-params['early_stopping_round'], 0] for dft in dfs)
                score = sum(scores) / len(scores)
                nboost = tuple(len(dft[1])-params['early_stopping_round'] for dft in dfs)
        self._save = save

## specs for search logic

### params searching specs

In [76]:
# class LPS(O()):
#     class Discrete(O()):
#         enc = {
#             'learning_rate': [.05],
#             ('max_depth','num_leaves'): #10
#                 [(6,1<<6),(8,1<<6),(8,1<<8),(10,1<<6),(10,1<<8),(10,1<<10),(12,1<<8),(12,1<<10),(12,1<<12),
#                  (-1,1<<8),(-1,1<<10),(-1,1<<12),(-1,1<<14)],
#             #('max','num'): #10
#             #    [(6,6),(9,7),(9,9),(12,8),(12,10),(12,12),(-1,8),(-1,10),(-1,12),(-1,14)],
#         }
        
#     class OneByOne(O()):
#         class info(O()):
#             a = "main [a]rray data of 3 values, [min mid max]"
#             b =  "[b]ack up value, i.e. default value if array doesn't give better results"
#             cast = "function to apply to values before using to cast to the right dtype"
#             lim = "maximum number of iterations of searching in this hyperparameter"
#         class default(O()):
#             cast = keepSigFig(2)
#             lim = 2
#         data = {
#             'min_data_in_leaf': O(a=[1,60,375], cast=round),
#             'min_sum_hessian_in_leaf': O(a=[0,50,200]),
#             'lambda_l1': O(a=[0,.02,.2], b=0, lim=1),
#             'lambda_l2': O(a=[0,.02,.2], b=0, lim=1),
#         }

### model search specs

In [77]:
# class LMS(O()):
#     model = O(
#         time = 'time',
#         value = 'y',
#         target = 'target',
#         weight = 'weight1',
#     )
    
#     metric = KaggleMetric()
#     search = LPS
    
#     class Features(O()):
#         '''features selection groups'''
#         data = [
#             ['f1','f2','f3'],
#             ['f4','f5','f6'],
#         ]
    
#     class Samples(O()):
#         '''sample learning/cv split'''
#         # O(method='GroupShuffleSplit.2', kwargs=dict(n_splits=<(5)many>, test_size=.5, random_state=44), groups='quarter'),
#         enc = [
#             O(method='group.2', groups='quarter', data=[
#                 [2009.0,2009.75,2010.75,2011.5,2012.25,2012.5,2012.75,2013.0,2013.25,2013.75,2014.0,2014.25,2015.0],
#                 [2009.25,2009.5,2010.0,2010.25,2010.5,2011.0,2011.25,2011.75,2012.0,2013.5,2014.5,2014.75,2015.25]
#             ]),
#             O(method='group.2', groups='quarter', data=[
#                 [2009.25,2009.5,2011.0,2011.25,2011.5,2011.75,2012.0,2012.25,2013.0,2013.5,2014.5,2014.75,2015.0],
#                 [2009.0,2009.75,2010.0,2010.25,2010.5,2010.75,2012.5,2012.75,2013.25,2013.75,2014.0,2014.25,2015.25]
#             ]),
#             O(method='group.2', groups='quarter', data=[
#                 [2011.0,2011.75,2012.0,2012.25,2012.75,2013.25,2013.5,2014.0,2014.25,2014.5,2014.75,2015.0,2015.25],
#                 [2009.0,2009.25,2009.5,2009.75,2010.0,2010.25,2010.5,2010.75,2011.25,2011.5,2012.5,2013.0,2013.75]
#             ]),
#             O(method='group.2', groups='quarter', data=[
#                 [2009.5,2010.0,2010.25,2010.5,2011.0,2011.75,2012.25,2012.5,2013.0,2013.5,2014.25,2014.75,2015.25],
#                 [2009.0,2009.25,2009.75,2010.75,2011.25,2011.5,2012.0,2012.75,2013.25,2013.75,2014.0,2014.5,2015.0]
#             ]),
#             O(method='group.2', groups='quarter', data=[
#                 [2009.75,2010.0,2010.25,2010.75,2011.75,2012.0,2012.25,2012.5,2013.0,2013.5,2013.75,2014.5,2015.0],
#                 [2009.0,2009.25,2009.5,2010.5,2011.0,2011.25,2011.5,2012.75,2013.25,2014.0,2014.25,2014.75,2015.25]
#             ]),
#             O(method='group.2', groups='quarter', data=[
#                 [2009.25,2009.5,2010.0,2010.25,2010.5,2010.75,2011.5,2013.0,2013.5,2014.0,2014.5,2014.75,2015.25],
#                 [2009.0,2009.75,2011.0,2011.25,2011.75,2012.0,2012.25,2012.5,2012.75,2013.25,2013.75,2014.25,2015.0]
#             ]),
#         ]
        
#     class Params(O()):
#         '''parameters constant settings'''
#         data = dict(
#             objective = 'binary',
#             num_iterations = 100000,
#             early_stopping_round = 50,
#             metric = 'None',
#             seed = 44,
#             bagging_seed = 45,
#             feature_fraction_seed = 46,
#         )

# -=#=-=#=-=#=-=#=-=#=- Features selection logic -=#=-=#=-=#=-=#=-=#=-=#=-

### -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=- Abstract Function datatype -=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-

In [78]:
def item_check(a):
    if not isinstance(a, tuple):
        a = (a,)
    return a

class Itemgetter():
    def __init__(self, func):
        self.func = func
    def __getitem__(self, a):
        return self.func(item_check(a))

In [79]:
# testing
if TEST:
    guy = Itemgetter(lambda *a: SEE(*a))
    guy[5.7,8]

In [80]:
# testing yo!
if TEST:
    class Yo():
        def __call__(self, x):
            SEE(x, '!')
    yo = Yo()
    yo.__call__ = lambda x: SEE('fuck', x)
    yo(5)

#### Function super useful datatype

In [81]:
class Function(type):
    def __new__(self, *args, **kw):
        noattach = '_noattach_' in kw
        if len(args) == 1:
            if isinstance(args[0], __class__):
                return args[0]
            assert callable(args[0])
            name = args[0].__qualname__
            parents = ()
            attrs = {'__call__': args[0]}
            noattach = True
        else:
            name, parents, attrs = args
        for k, v in list(dict.items(attrs)):
            if not isinstance(v, classmethod) and callable(v) and not isinstance(v, __class__) and not noattach:
                attrs[k] = classmethod(v)
            
        return super().__new__(self, name, parents, attrs)
    
    def __init__(self, *args, **kw):
        #SEE('init', args, kw)
        if len(args) == 1:
            return
        name, parents, attrs = args
        for k in attrs:
            v = getattr(self, k)
            #SEE('init ->', k, v)
            if k[:2] != '__' and callable(v):
                setattr(self, k, __class__(v, _noattach_=True))
        
    def __call__(self, *a, **k):
        return self.__call__(*a, **k)
        
    def __getitem__(self, *a, **k):
        return self.__getitem__(*a, **k)
    
    __inv__ = lambda f: __class__(lambda *a, **k: ~f(*a,**k))
    __and__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) & g(*a,**k))
    __or__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) | g(*a,**k))
    __xor__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) ^ g(*a,**k))

In [82]:
# testing
if TEST:
    class What(metaclass=Function):
        def __call__(self, x, y):
            return np.sqrt(x**2 + y**2)
        def hi(self, *s):
            SEE('hi, ' + s[0])
        __getitem__ = lambda self, a: lambda arg: SEE(arg, arg*a)
    What.hi('u')
    What.hi#.__call__
    What[5](3)

#### Function query language implementation

In [83]:
def scan(s):
    s = s.replace(' ','').replace('\n','').replace('\t','').replace('\r','') + '\0'
    i0, i = 0, 0
    while i0 < len(s)-1 and i < len(s):
        c = s[i]
        if 'a' <= c <= 'z' or 'A' <= c <= 'Z' or '0' <= c <= '9' or c == '_':
            i += 1
            continue
        elif c == '\0':
            yield s[i0:i]
            break
        elif c in '~&|^()':
            if i0 < i:
                yield s[i0:i]
            yield c
            i0 = i = i+1
        elif c in '[{':
            if i0 < i:
                yield s[i0:i]
            i0 = i
            match = ']' if c=='[' else '}'
            while s[i] != match:
                i += 1
            i += 1
            yield s[i0:i]
            i0 = i
        else:
            raise AssertionError('bad syntax in input to scanner')

            
Token = namedtuple('Token', 'type value')

def tokenize(s):
    prevIsField = False
    for w in s:
        a = w[0]
        if 'a' <= a <= 'z' or 'A' <= a <= 'Z' or '0' <= a <= '9' or a == '_':
            yield Token('field', w)
            prevIsField = True
            continue
        elif w in ['&','^','|']:
            yield Token('op2', w)
        elif w in ['~']:
            yield Token('op1', w)
        elif w in ['(']:
            yield Token('_group', w)
        elif w in [')']:
            yield Token('group_', w)
        elif a == '{':
            assert prevIsField, '{} section must have been preceded by a field name'
            yield Token('op2', 'query')
            yield Token('raw', w)
        elif a == '[':
            if not prevIsField:
                yield Token('self', '')
            yield Token('op2', 'getitem')
            yield Token('raw', w)
        else:
            assert False, f'invalid word encountered in tokenizer input: {w}'
        prevIsField = False
        
class TokenInfo(O()):
    self = 'self, the owning Function instance'
    field = 'field name of Function instance'
    op2 = 'binary operator (look in the map)'
    op1 = 'unary operator (look in the map)'
    _group = "open grouping (just '(')"
    group_ = "close grouping (just ')')"
    raw = "raw data to be fed to operator"


def parse(s):
    stack = []
    for t in s:
        if t.type in ['field', 'self', 'raw']:
            yield t
        elif t.type in ['op2', 'op1']:
            while stack and stack[-1].type in ['op2', 'op1'] and parse.order[t.value] >= parse.order[stack[-1].value]:
                yield stack.pop()
            stack.append(t)
        elif t.type == '_group':
            stack.append(t)
        elif t.type == 'group_':
            while True:
                if not stack:
                    raise AssertionError('too many close parentheses')
                x = stack.pop()
                if x.type == '_group':
                    break
                else:
                    assert x.type in ['op2', 'op1'], 'internal error'
                    yield x
    while stack:
        x = stack.pop()
        if x.type == '_group':
            raise AssertionError('too many open parentheses')
        assert x.type in ['op2', 'op1'], 'internal error'
        yield x
parse.order = {'query': 0, 'getitem': 0, '~': 1, '&': 2, '^': 3, '|': 4}

    
def query(self, qs):
    revpol = list(parse(tokenize(scan(qs))))
    stack = []
    for t in revpol:
        if t.type == 'self':
            stack.append(self)
        elif t.type == 'field':
            stack.append(getattr(self, t.value))
        elif t.type == 'raw':
            stack.append(t.value)
        elif t.type == 'op1':
            a = stack.pop()
            x = query.opTbl[t.value](a)
            stack.append(x)
        elif t.type == 'op2':
            b, a = stack.pop(), stack.pop()
            x = query.opTbl[t.value](a, b)
            stack.append(x)
    return stack.pop()

class GetItemHelper(metaclass=Singleton):
    def __getitem__(self, a):
        return a

query.opTbl = {
    '~': operator.__inv__,
    '&': operator.__and__,
    '^': operator.__xor__,
    '|': operator.__or__,
    'query': lambda f, q: f.query(q[1:-1]), #TODO can only go one level deep
    'getitem': lambda f, a: f[eval(f'GetItemHelper()[{a[1:-1]}]')]
}

Function.query = query

In [84]:
# testing function query
if TEST:
    qs = 'what2{f_ck} & no_[5,62,6:6]  |  [9] & ~(ho_10|[3,5:9])'
    scanned = list(scan(qs))
    assert (scanned == ['what2','{f_ck}','&','no_','[5,62,6:6]','|','[9]','&','~','(','ho_10','|','[3,5:9]',')']), (
            'Unit test fail for scanner')    
    tokens = list(tokenize(scanned)) #TODO make unit test
    parsed = list(parse(tokens)) # TODO make unit test
    SEE(qs)
    SEE('-'*99)
    SEE(scanned)
    SEE('-'*99)
    for line in tokens:
        SEE(line)
    SEE('-'*99)
    for line in parsed:
        SEE(line)

## .!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!. The actual Features subsets definitions!!!! .!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.

In [85]:
def FFF__returns_range(x, e):
    if e == 0:
        ret = x in 'oo oo0 cc cc0 (oo-oo0) (cc-cc0) (cc-cc0){0} (oo-oo0){0} (oo0-cc0) it af'.split()
        ret |= '(it,af,it{1},oo0,cc0)' in x
        return ret
    elif e == 1 and '(it{1},af{1},it{2},oo0{1},cc0{1})' in x:
        return True
    elif isinstance(e, int):
        suf = '{' + str(e) + '}'
        return x[-len(suf):] == suf
    elif isinstance(e, slice):
        start, stop = e.start if e.start is not None else 0, e.stop if e.stop is not None else 0
        assert start >= 0 and stop >= 0
        sml, lrg = min(start, stop), max(start, stop)
        ret = ('(it,af,it{1},oo0,cc0){0/1}' not in x and '(it{1},af{1},it{2},oo0{1},cc0{1})' not in x and
            '({}..{})'.format(sml,lrg-1) in x or '{{{}/{}}}'.format(sml,lrg-1) in x)
        suf1, suf2 = 'Since'+str(lrg), '_'+str(lrg)
        ret |= x[-len(suf1):] == suf1 and sml == 0
        ret |= x[-len(suf2):] == suf2 and sml == 0
        return ret
    return False
    
def FFF__returns_span(x, e):
    if isinstance(e, int):
        assert e > 0
    suf1, suf2 = 'Since'+str(e), '_'+str(e)
    if x[-len(suf1):] == suf1 or x[-len(suf2):] == suf2:
        return True
    if e == 1:
        return any(FFF__returns_range(x, r) for r in [0,1,2,3,4,5])
    elif e == 2:
        return (x[-5:] == '{0/1}' or x[-5:] == '{1/2}') and ',' not in x
    elif e == 3:
        return x[-5:] == '{0/2}' and ',' not in x
    elif e == 5:
        return '(0..4)' in x or '(5..9)' in x or '(10..14)' in x or '(15..19)' in x
    elif e == 10:
        return '(0..9)' in x or '(5..14)' in x or '(10..19)' in x
    elif e == 15:
        return '(0..14)' in x or '(5..19)' in x
    elif e == 20:
        return '(0..19)' in x
    elif isinstance(e, slice):
        start, stop = e.start if e.start is not None else 0, e.stop if e.stop is not None else 0
        assert start >= 0 and stop >= 0
        sml, lrg = min(start, stop), max(start, stop)
        for i in [1,2,3,4,5,10,15,20,21,40,60,62,120,125,250]:
            if sml < i <= lrg and FFF__returns_span(x, i):
                return True
    return False
    

class FFF(metaclass=Function):
    #class Basic(metaclass=Function):
    #    pass
    
    class AssetEnc(metaclass=Function):
        __call__ = lambda f, x: 'assetCode' in x or 'assetName' in x
        Ids = lambda f, x: x in ['assetCodeId', 'assetNameId']
        
        class RandMap(metaclass=Function):
            __call__ = lambda f, x: 'randMap32' in x
            squared = Itemgetter(lambda a: lambda x:
                any(x[::-1].find(('randMap32_' + str(e**2))[::-1]) == 0 for e in item_check(a)))
            __getitem__ = (lambda f, a: lambda x:
                any(x[::-1].find(('randMap32_' + str(e))[::-1]) == 0 for e in item_check(a)))
        
        WTFF = lambda f, x: 'WTFF' in x
        InUni = lambda f, x: 'inUniCount' in x
    
    class Time(metaclass=Function):
        __call__ = lambda f, x: x in ['dayOfYear', 'dayOfWeek']
        
    class Volatility(metaclass=Function):
        __call__ = lambda f, x: 'volatility' in x or 'Volatility' in x
        __getitem__ = lambda f, a: lambda x: any('olatility'+str(e) in x for e in item_check(a))
        
    class Adjustment(metaclass=Function):
        __call__ = lambda f, x: 'adjustmentMean' in x
        __getitem__ = lambda f, a: lambda x: any('adjustmentMean'+str(e) in x for e in item_check(a))
    
    class Drawdown(metaclass=Function):
        __call__ = lambda f, x: 'drawdown' in x or 'Drawdown' in x
        __getitem__ = lambda f, a: lambda x: (
            any((
                 'rawdown'+str(e) in x if isinstance(e, int) else
                 'rawdown({}-{})'.format(max(e.start,e.stop),min(e.start,e.stop)) in x if isinstance(e, slice) else
                 1/0
                ) for e in item_check(a))
        )
    
    class Drawup(metaclass=Function):
        __call__ = lambda f, x: 'drawup' in x or 'Drawup' in x
        __getitem__ = lambda f, a: lambda x: any(
            (
                'rawup'+str(e) in x if isinstance(e, int) else
                'rawup({}-{})'.format(max(e.start,e.stop),min(e.start,e.stop)) in x if isinstance(e, slice) else
                1/0
            ) for e in item_check(a)
        )
    
    class Price(metaclass=Function):
        __call__ = lambda f, x: x in ['open', 'close']
    
    class Level(metaclass=Function):
        __call__ = lambda f, x: x in ['open', 'close', 'volume']
    
    class Return(metaclass=Function):
        allow = lambda f, x: all(b not in x for b in
                                 ['MinSince','MaxSince','rawdown','rawup','ecordedDays','assetCode','volume','olatility'])
        __call__ = lambda f, x: 'oo' in x or 'cc' in x or x[:2] in ['it', 'af']
        oo = lambda f, x: 'oo' in x and not any(['oo0' in x, '-oo' in x, 'oo-' in x, ',oo' in x, 'oo,' in x])
        oo0 = lambda f, x: 'oo0' in x and not any(['-oo0' in x, 'oo0-' in x, ',oo0' in x, 'oo0,' in x])
        cc = lambda f, x: 'cc' in x and not any(['cc0' in x, '-cc' in x, 'cc-' in x, ',cc' in x, 'cc,' in x])
        cc0 = lambda f, x: 'cc0' in x and not any(['-cc0' in x, 'cc0-' in x, ',cc0' in x, 'cc0,' in x])
        doo = lambda f, x: '(oo-oo0)' in x
        dcc = lambda f, x: '(cc-cc0)' in x
        doc0 = lambda f, x: '(oo0-cc0)' in x
        it = lambda f, x: x[:3] in ['it', 'it{']
        af = lambda f, x: x[:3] in ['af', 'af{']
        mix = lambda f, x: x[:3] == '(it'
        
        def __getitem__(f, a):
            return lambda x: any(FFF__returns_range(x, e) for e in item_check(a))
        
        span = Itemgetter(lambda a: lambda x: any(FFF__returns_span(x, e) for e in item_check(a)))
                
    class Since(metaclass=Function):
        __call__ = lambda f, x: 'Since' in x
        __getitem__ = lambda f, a: lambda x: any('Since' + str(e) in x for e in item_check(a))
    
    class Volume(metaclass=Function):
        __call__ = lambda f, x: x == 'volume'
    
    class VolumeRatioMean(metaclass=Function):
        '''~Back2 ~Back2{1} ~5 ~10 ~10{5} ~5ByMean20'''
        __call__ = lambda f, x: 'olumeRatioMean' in x
        __getitem__ = lambda f, a: lambda x: x in ['volumeRatioMean'+str(e) for e in item_check(a)]
        
    class FracRec(metaclass=Function):
        __call__ = lambda f, x: 'fracRecordedDays' in x
        __getitem__ = lambda f, a: lambda x: any(x == 'fracRecordedDaysSince'+str(e) for e in item_check(a))

FFF.Global = FFF.Volatility | FFF.Adjustment

In [86]:
#pd.to_pickle((F,P), '/big/data/saves/train_5searchbase.32.pkl')
F,P = pd.read_pickle('/big/data/saves/train_5searchbase.32.pkl')
pd.set_option('display.max_rows', 500)

In [87]:
#json.dump(list(F.columns), open('feats.json', 'w'))
feats = json.load(open('feats.json', 'r'))

In [88]:
# testing my operators and query language
#pd.Series(F.columns.map(FFF.FracRec[125]), index=F.columns)
#pd.Series(F.columns.map(FFF.query('Return{allow} & Return{span[10:]}')), index=F.columns)

In [89]:
list(filter(lambda x: x[:2]!='__', dir(FFF)))

['Adjustment',
 'AssetEnc',
 'Drawdown',
 'Drawup',
 'FracRec',
 'Global',
 'Level',
 'Price',
 'Return',
 'Since',
 'Time',
 'Volatility',
 'Volume',
 'VolumeRatioMean']

In [90]:
fffList = [
    FFF.Return & FFF.Return.oo0,
    FFF.Drawdown
]

In [91]:
class LPS(O()):
    class Discrete(O()):
        enc = {
            'learning_rate': [.05],
            ('max_depth','num_leaves'): #10
                [(6,1<<6),(8,1<<6),(8,1<<8),(10,1<<6),(10,1<<8)],
                 #,(10,1<<10),(12,1<<8),(12,1<<10),(12,1<<12),
                 #(-1,1<<8),(-1,1<<10),(-1,1<<12),(-1,1<<14)],
            #('max','num'): #10
            #    [(6,6),(9,7),(9,9),(12,8),(12,10),(12,12),(-1,8),(-1,10),(-1,12),(-1,14)],
        }
        
    class OneByOne(O()):
        class info(O()):
            a = "main [a]rray data of 3 values, [min mid max]"
            b =  "[b]ack up value, i.e. default value if array doesn't give better results"
            cast = "function to apply to values before using to cast to the right dtype"
            lim = "maximum number of iterations of searching in this hyperparameter"
        class default(O()):
            cast = keepSigFig(2)
            lim = 1
        data = {
            'min_data_in_leaf': O(a=[1,60,375], cast=round),
            'min_sum_hessian_in_leaf': O(a=[0,50,200]),
            'lambda_l1': O(a=[0,.02,.2], b=0, lim=0),
            'lambda_l2': O(a=[0,.02,.2], b=0, lim=0),
        }

class LMS(O()):
    model = O(
        time = 'time',
        value = 'y',
        target = 'target',
        weight = 'weight1',
    )
    
    metric = KaggleMetric()
    search = LPS
    
    class Features(O()):
        '''features selection groups'''
        data = [list(filter(fff, feats)) for fff in fffList]
    
    class Samples(O()):
        '''sample learning/cv split'''
        # O(method='GroupShuffleSplit.2', kwargs=dict(n_splits=<(5)many>, test_size=.5, random_state=44), groups='quarter'),
        enc = [
            O(method='group.2', groups='quarter', data=[
                [2009.0,2009.75,2010.75,2011.5,2012.25,2012.5,2012.75,2013.0,2013.25,2013.75,2014.0,2014.25,2015.0],
                [2009.25,2009.5,2010.0,2010.25,2010.5,2011.0,2011.25,2011.75,2012.0,2013.5,2014.5,2014.75,2015.25]
            ]),
            O(method='group.2', groups='quarter', data=[
                [2009.25,2009.5,2011.0,2011.25,2011.5,2011.75,2012.0,2012.25,2013.0,2013.5,2014.5,2014.75,2015.0],
                [2009.0,2009.75,2010.0,2010.25,2010.5,2010.75,2012.5,2012.75,2013.25,2013.75,2014.0,2014.25,2015.25]
            ]),
#             O(method='group.2', groups='quarter', data=[
#                 [2011.0,2011.75,2012.0,2012.25,2012.75,2013.25,2013.5,2014.0,2014.25,2014.5,2014.75,2015.0,2015.25],
#                 [2009.0,2009.25,2009.5,2009.75,2010.0,2010.25,2010.5,2010.75,2011.25,2011.5,2012.5,2013.0,2013.75]
#             ]),
#             O(method='group.2', groups='quarter', data=[
#                 [2009.5,2010.0,2010.25,2010.5,2011.0,2011.75,2012.25,2012.5,2013.0,2013.5,2014.25,2014.75,2015.25],
#                 [2009.0,2009.25,2009.75,2010.75,2011.25,2011.5,2012.0,2012.75,2013.25,2013.75,2014.0,2014.5,2015.0]
#             ]),
#             O(method='group.2', groups='quarter', data=[
#                 [2009.75,2010.0,2010.25,2010.75,2011.75,2012.0,2012.25,2012.5,2013.0,2013.5,2013.75,2014.5,2015.0],
#                 [2009.0,2009.25,2009.5,2010.5,2011.0,2011.25,2011.5,2012.75,2013.25,2014.0,2014.25,2014.75,2015.25]
#             ]),
#             O(method='group.2', groups='quarter', data=[
#                 [2009.25,2009.5,2010.0,2010.25,2010.5,2010.75,2011.5,2013.0,2013.5,2014.0,2014.5,2014.75,2015.25],
#                 [2009.0,2009.75,2011.0,2011.25,2011.75,2012.0,2012.25,2012.5,2012.75,2013.25,2013.75,2014.25,2015.0]
#             ]),
        ]
        
    class Params(O()):
        '''parameters constant settings'''
        data = dict(
            objective = 'binary',
            num_iterations = 100000,
            early_stopping_round = 50,
            metric = 'None',
            seed = 44,
            bagging_seed = 45,
            feature_fraction_seed = 46,
        )

In [92]:
mm = ModelManager('/big/data/search/fake')
ms = ModelSearch(specs=LMS, mm=mm, X=F, Y=P)
walk = ms.walk()

In [93]:
i = -1

In [94]:
%%time
i += 1
print((i,) + next(walk))

(0, (1, 0), <>(train=[0.7214605834581501, 1.073667926072597], scores=[0.5675374430153964, 0.5639808673520671], score=0.5657591551837318, nboost=[245, 106]))
CPU times: user 25 s, sys: 3.32 s, total: 28.3 s
Wall time: 5.8 s


# ================= ACTUAL PRODUCTION RUN ====================

# ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,, Live Run Testing (still, just testing) ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

### making test variables

In [81]:
from scipy.stats import norm
ppf = norm.ppf
eps = np.finfo(float).eps

In [82]:
def make_new_vars():
    n = 1000
    ##################RANDOMIZE#######################
    while True:
        try:
            u = np.random.uniform(size=(5, n))
            assert (u!=0).all()
            break
        except AssertionError:
            continue
    uu = u.copy()
    uu[0][np.random.randint(252, size=n) == 0] = np.nan
    uu[1][np.random.randint(173, size=n) == 0] = np.nan
    uu[2][np.random.randint(81, size=n) == 0] = np.nan
    uu[3][np.random.randint(27, size=n) == 0] = np.nan
    #################ENDRANDOM######################
    F = pd.DataFrame({'hax': np.arange(n)})
    F['time'] = F.hax // 10
    F['assetCodeId'] = F.hax % 10
    F['quarter'] = F.hax // 100
    P = F.copy()
    F['alex'] = ppf(u[0]*5%.99+eps)*ppf(u[1]*7%.99+eps)
    F['bob'] = ppf(.5+.49*uu[2]*np.sin(F.assetCodeId.values**10))*np.exp(uu[3])
    F['carol'] = -np.log(u[0]) + 4 * np.sin(1/uu[1])
    F['dean'] = u[1] + 2 * uu[2] - np.exp(-uu[3])
    F['edgar'] = ppf( (uu[2]**2-2*u[1]+(u[0]-u[3])**3-.5*u[1]**2) % .99 + eps )
    P['y'] = (ppf((u[0]+u[1])/2) + ppf((u[2]+u[3])/2) + .2*ppf(u[4])) * 1e-1
    P['universe'] = (~np.isnan(uu[0])).astype(float)
    return F, P

In [83]:
#F, P = make_new_vars()
#Path('lighttest.pkl').write_bytes(pickle.dumps((F, P)))
F, P = pickle.loads(Path('lighttest.pkl').read_bytes())

In [84]:
P['target'] = P.y>0
P['upDown'] = (P.target*2-1)
P['upDown1'] = P.upDown*P.universe.astype(int)
P['absVal'] = np.abs(P.y)
P['absVal1'] = P.absVal*P.universe
P['weight'] = P.absVal#.qtl()
P['weight1'] = P.weight*P.universe

In [85]:
F.head()

,hax,time,assetCodeId,quarter,alex,bob,carol,dean,edgar
0,0,0,0,0,-0.012198,0.000000,3.655924,1.336200,0.601672
1,1,0,1,0,-0.033782,1.139956,8.008799,1.244428,0.349115
2,2,0,2,0,0.034335,-0.333275,2.013295,1.549419,1.363992
3,3,0,3,0,-2.131567,-0.049046,5.325198,0.450859,0.940187
4,4,0,4,0,1.636082,0.267445,4.698875,0.670703,-0.449314


In [86]:
P.head()

,hax,time,assetCodeId,quarter,y,universe,target,upDown,upDown1,absVal,absVal1,weight,weight1
0,0,0,0,0,0.076738,1.0,True,1,1,0.076738,0.076738,0.076738,0.076738
1,1,0,1,0,0.048210,1.0,True,1,1,0.048210,0.048210,0.048210,0.048210
2,2,0,2,0,0.089397,1.0,True,1,1,0.089397,0.089397,0.089397,0.089397
3,3,0,3,0,-0.028319,1.0,False,-1,-1,0.028319,0.028319,0.028319,0.028319
4,4,0,4,0,-0.103184,1.0,False,-1,-1,0.103184,0.103184,0.103184,0.103184


In [87]:
class LPS(O()):
    class Discrete(O()):
        enc = {
            'learning_rate': [.05],
            ('max_depth','num_leaves'):
                [(3,1<<3),(6,1<<6),(-1,1<<9)]
        }
        
    class OneByOne(O()):
        class info(O()):
            a = "main [a]rray data of 3 values, [min mid max]"
            b =  "[b]ack up value, i.e. default value if array doesn't give better results"
            cast = "function to apply to values before using to cast to the right dtype"
            lim = "maximum number of iterations of searching in this hyperparameter"
        class default(O()):
            cast = keepSigFig(2)
            lim = 2
        data = {
            'min_data_in_leaf': O(a=[1,6,37], cast=round),
            'min_sum_hessian_in_leaf': O(a=[0,5,20], b=0),
            'lambda_l1': O(a=[0,.02,.2], b=0, lim=1),
            'lambda_l2': O(a=[0,.02,.2], b=0, lim=1),
        }


class LMS(O()):
    model = O(
        time = 'time',
        value = 'y',
        target = 'target',
        weight = 'weight1',
    )
    
    metric = KaggleMetric()
    search = LPS
    
    class Features(O()):
        '''features selection groups'''
        data = [
            ['alex', 'bob', 'carol'],
            ['bob', 'dean', 'edgar'],
            ['alex', 'carol', 'edgar']
        ]
    
    class Samples(O()):
        '''sample learning/cv split'''
        enc = [
            O(method='group.2', groups='quarter', data=[
                [0, 1, 2, 3, 4],
                [5, 6, 7, 8, 9]
            ]),
        ]
        
    class Params(O()):
        '''parameters constant settings'''
        data = dict(
            objective = 'binary',
            num_iterations = 100000,
            early_stopping_round = 50,
            metric = 'None',
            seed = 44,
            bagging_seed = 45,
            feature_fraction_seed = 46,
        )

In [88]:
mm = ModelManager('/big/data/search/test')
ms = ModelSearch(specs=LMS, mm=mm, X=F, Y=P)
walk = ms.walk()

In [89]:
i = -1

In [93]:
i += 1
print((i,) + next(walk))

training...
(3, (0, 0), <>(train=(1.9117454484505862, 1.8610316178296742), scores=(1.7104576224320605, 1.7875447679802607), score=1.7490011952061606, nboost=(93, 128)))


# -.-.-.-.-.-.-.-.-.-.-.-.-.- old shit / testing -.-.-.-.-.-.-.-.-.-.-.-.-.-.-

In [105]:
# PUREL TESTING
class HI(O()):
    class DERP():
        val = -99
        dir = Path('.')
        def yo(x):
            def y(a):
                return x(a) + 1000
            return y
        my = O(what=yo(lambda a: a ** 2))
        hm = O(func=lambda x: x * __class__.val, goto=lambda x: __class__.dir/x)
        class WHAT(O()):
            dude = lambda x: x + 10
        class YES(O()):
            dude = WHAT.dude
HI.DERP.my.what(5)
HI.DERP.hm.func(5)
HI.DERP.dir = Path('what')
HI.DERP.hm.goto('.git')
HI.DERP.WHAT.dude(0)

# ANOTHER PURE TESTING
def dothe(self):
    self.val = 4
    top = 100
    class Derp(O()):
        def func(x):
            return x + self.val
        def top(y):
            return y * top
    self.derp = Derp
item = Stop()
dothe(item)
item.derp.top(3)

class Yo(O()):
    class What(O()):
        derp = 5
    class Another(O()):
        yep = What.derp
Yo()

NameError: name 'What' is not defined